In [1]:
import numpy as np
import scipy as sp
from scipy import signal
import librosa
import matplotlib.pyplot as plt
import pywt
import sounddevice as sd
import math

In [2]:
#Επειδη η librosa δεν μπορει να υποστηριξει mp3 files τα μετατρεπουμε σε wav

In [3]:
def audiosignal (signal, sample_rate) : 
    x,fs = librosa.load(signal, sample_rate)
    return x , fs

In [4]:

#Ερώτημα 4.1
sr=22050 #sampling frequency
foxtrot_1, fs = audiosignal('foxtrot_excerpt1.wav',sr) 
#Απομονώνονται 2^16 δείγματα από το αρχείο και εκτιμάται οπτικά η παλμική περιοδικότητα από τη γραφική του παράσταση.

In [5]:
foxtrot_signal_1 = foxtrot_1[(1*(2**16)):(2*(2**16))] 
sd.play(foxtrot_1,sr)

In [6]:
def signal_plot(signal,fs) :
    
    fig = plt.figure(figsize= (20,8))
    t=np.linspace(0, (len(signal))/fs, len(signal))
    plt.plot(t, signal)
    plt.xlabel('Time (sec)', fontsize=14)
    plt.ylabel('Amplitude', fontsize=14)
    plt.title('Music song', fontsize = 18)
    #figsalsa1.savefig("4.1.png")
    plt.show()

In [ ]:
signal_plot(foxtrot_signal_1, fs)

In [ ]:
#Οπτικά διακρίνουμε έναν ισχυρό παλμό ανά 0.25 second περίπου

In [ ]:
#Εφαρμόζουμε την πολυκλιμακωτή ανάλυση (σε 7 επίπεδα) του discrete wavelet transform χρησιμοποιόντας τη συνάρτηση pywt.wavedec και στη συνέχεια κα διαιρούμαι τα σήματα με ρίζα 2 ώστε να είναι όλα στην ίδια κλίμακα

In [ ]:
coefs = pywt.wavedec(foxtrot_signal_1, "db4", level=7)

In [ ]:
def normalization(coefs): 
    yi_norm = []
    for i in range(len(coefs)) :#διαίρεση με ρίζα 2 σε κάθε στάδιο
         
        if i==0 :
            yi_norm.append(coefs[i]/(np.sqrt(2)**7))
        
        else :
            yi_norm.append(coefs[i]/(np.sqrt(2)**(8-i))) 
    
    return yi_norm

In [ ]:
yi_norm = normalization(coefs)

In [ ]:
#Ερώτημα 4.3
#Για τον υπολογισμό του ερωτήματος ακολουθούνται οι οδηγίες της εκφώνησης. Αρχικά υπολογίζεται η απόλυτη τιμή του σήματος.

In [ ]:
def absolute (yi_norm):
    zi = []

    for y in yi_norm:
        zi.append(np.abs(y))
        
    return zi

In [ ]:
zi = absolute(yi_norm)

In [ ]:
#Στη συνέχεια, γίνεται χρήση βαθυπερατού φίλτρου για τις διάφορες τιμές του a

In [ ]:
def surround(zlist, a0) :
    
    surround = []
    
    for i in range(0,len(zlist)) :
        
        if i==0 :
            a = a0 * (2**7)
        
        else :
            a = a0 * (2**(8-i))
        
        x_i = np.zeros(len(zlist[i]))
        
        
        for j in range(1,len(zlist[i])):
            
            x_i[j] = (1-a)*x_i[j-1] + a*zlist[i][j]
            
        surround.append(x_i)
        
    return surround

In [ ]:
def list_a_surrounds(zi) :    
    
    list_a = [0.001, 0.002, 0.005]

    surrounds = []

    for i in list_a:

        current_surround = surround(zi, i)
        surrounds.append(current_surround)
    
    return surrounds

In [ ]:
surrounds = list_a_surrounds(zi)

In [ ]:
#Τέλος, κεντράρισμα του σήματος με αφαίρεση της μέσης τιμής του, η οποία δίνεται από τη συνάρτηση mean() της numpy.

In [ ]:
def central(list) :
    for i in range(len(list)) :
        for j in range(len(list[i])) :
            list[i][j] = list [i][j] - np.mean(list[i][j])
            
    return list

In [ ]:
centralized_surrounds = central(surrounds)

In [ ]:
#Παρακάτω παρατίθενται τα διαγράμματα των συνιστωσών yd3 και yd6 μαζί με τις περιβάλλουσές τους για τις διάφορες τιμές του a

In [ ]:
def coefficients_and_surrounds_plot (centralized_surrounds, yi_norm) : 
    plot_sur = []
    plot_y = []

    for i in [5, 2]:
        plot_sur.append([centralized_surrounds[0][i], centralized_surrounds[1][i], centralized_surrounds[2][i]])
        plot_y.append(yi_norm[i])

    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(13,18))
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=7.0)

    idx = 0

    for ax, sur, y in zip(axes.flatten(), plot_sur, plot_y):
        if idx == 0:
            title = "Η συνιστώσα y$_d$$_3$ και οι τρεις περιβάλλουσες"
            idx += 1
            axis_x = np.linspace(0,len(sur[0]), len(sur[0]))
            ax.plot(axis_x, y, label = "y$_d$$_3$")
            ax.plot(axis_x, sur[0], label = "a$_0$ = 0.001")
            ax.plot(axis_x, sur[1], label = "a$_0$ = 0.002")
            ax.plot(axis_x, sur[2], label = "a$_0$ = 0.005")
        else:
            title = "Η συνιστώσα y$_d$$_3$ και οι τρεις περιβάλλουσες"
            axis_x = np.linspace(0,len(sur[0]), len(sur[0]))
            ax.plot(axis_x, y, label = "y$_d$$_3$")
            ax.plot(axis_x, sur[0], label = "a$_0$ = 0.001")
            ax.plot(axis_x, sur[1], label = "a$_0$ = 0.002")
            ax.plot(axis_x, sur[2], label = "a$_0$ = 0.005")

        ax.legend(shadow=True, fontsize='x-large',)
        ax.set_title(title, fontsize=18)
        ax.set_xlabel("n", fontsize=16)

    
    plt.show()

In [ ]:
coefficients_and_surrounds_plot (centralized_surrounds, yi_norm)

In [ ]:
#Όπως ήταν αναμενόμενο οι περιβάλλουσες ακολουθούν σε γενικές γραμμές το σχήμα του σήματος αλλά είναι χαμηλωμένες αφού τους έχει αφαιρεθεί ο μέσος όρος. Επίσης, συμπαιρένουμε ότι η ακριβέστερη περιβάλλουσα προκύπτει για a=0.005 και για αυτή θα γίνει στη συνέχεια η αυτοσυσχέτιση του σήματος

In [ ]:
#Ερώτημα 4.4
#Για την πρόσθεση των περιβαλλουσών των συνιστωσών του προηγούμενου ερωτήματος:

In [ ]:
def add_surround (list, idx) :
    length = len(list[idx][7])
    
    current_list = list[idx]
    
    add=np.zeros(length)
    
    for sur in current_list :
        
        x=np.linspace(0, len(sur), length)
        x_p = np.arange(0,len(sur))
        interpolate = np.interp(x, x_p, sur) 
        
        add = add + interpolate
        
    return add

In [ ]:
sum_surround = add_surround(centralized_surrounds, 2)

In [ ]:
#Τώρα θα υπολογίσουμε την αυτοσυσχέτιση του σήματος (δηλαδή τι σχέση έχει ένα δείγμα με τα προηγούμενα του δείγματα) με χρήση της συνάρτησης correlate της numpy

In [ ]:
def autocorr(x):
   
    result = np.correlate(x, x, mode='full')
    
    return result[int(result.size/2):]


In [ ]:
def autocorr_plot(sum_surround) :
    
    autocorrelation = autocorr(sum_surround)

    t= np.arange(0,len(autocorrelation)-1)
    fig = plt.figure(figsize = (20,8))
    plt.plot(t,(autocorrelation[0:int(len(autocorrelation) - 1)]))
    plt.title("Autocorrelation", fontsize=18)
    plt.xlabel("Lags", fontsize=16)
    plt.ylabel
    
    return autocorrelation

In [ ]:
autocorrelation = autocorr_plot(sum_surround)

In [ ]:
#Ερώτημα 4.5
#Μέσω της αυτοσυσχέτισης φτιάξαμε το διάγραμμα του κομματιού σε Beats per minute και μέσω της συνάρτησης argrelextrema της scipy εντοπίσαμε το μέγιστο του διαγράμματος από το οποίο πήραμε το Bpm του κομματιού

In [ ]:
def make_bpm_plot (autoc, fs, signal):
    #autoc το autocorrelation κάθε σήματος
    max_autocorrelation = sp.signal.argrelextrema(autoc, np.greater)
    
    max_autoc = autoc[max_autocorrelation[0]]
    max_bpm = (60*fs)/max_autocorrelation[0]
    left = max_autocorrelation[0][0]
    right = max_autocorrelation[0][len(max_autocorrelation[0])-1]
    
    autocorrelation = (autoc[left:right])
    BPM = ((60*fs)/((np.arange(left,right))))
    
    left_limit = int((60*fs)/200)
    right_limit = int((60*fs)/40)
    max_index = left_limit + np.argmax(autoc[left_limit:right_limit])
    max_value = np.max (autoc[left_limit:right_limit])
    tempo = 60/(max_index/fs)
    
    fig = plt.figure(figsize=(20,8))
    plt.plot(BPM,autocorrelation)
    plt.plot(max_bpm, max_autoc, 'ro', label = "Local Maxima")
    plt.ylabel ("Beat strenght", fontsize = 16)
    plt.xlabel ("BPM", fontsize = 16)
    plt.vlines (40, ymin = np.min(autocorrelation), ymax = np.max(autocorrelation), color = "black")
    plt.vlines (200, ymin = np.min(autocorrelation), ymax = np.max(autocorrelation), color = "black")
    plt.title("Beat Histogram", fontsize = 20)
    plt.legend(shadow = True, loc = "lower right")
    plt.ylim(0)
    plt.xlim(0,250)
    
    plt.show()
    
    true_tempo = librosa.beat.tempo(signal,fs)
    print("Ο ρυθμός που βρήκαμε είναι:", tempo, "BPM")
    print("Ο κανονικός ρυθμός του τραγουδιού είναι :", true_tempo[0], "BPM")
    
    return tempo

In [ ]:
tempo = make_bpm_plot(autocorrelation,fs, foxtrot_signal_1)

In [ ]:
#Λάβαμε υπ' όψιν μας μόνο τα BPM που βρίσκονται μεταξύ 40 και 200 (που είναι και οι δύο κατακόρυφες γραμμές) καθώς οι κορυφές σε μεγαλύτερα BPM αποτελούν ακέραια πολλαπλάσια προήγουμενων BPM. To tempo αντιστοιχεί στα BPM με την υψηλότερη κορυφή. Με χρήση της συνάρτησης librosa.beat.tempo υπολογίσαμε και το πραγματικό tempo αυτού του κομματιού και όπως φαίνεται παραπάνω δεν έχει μεγάλη απόκλιση από αυτό που βρήκαμε

In [ ]:
def plot_tempo (signal , tempo, fs) :
    
    T = len(signal)/float(fs)
    beatfrequency = 60.0 / tempo
    beats = np.arange(0, T, beatfrequency)
    x=np.linspace(0, (len(signal))/fs, len(signal))
    
    plt.figure(figsize = (16,8))
    t=np.linspace(0, (len(signal))/fs, len(signal))
    plt.plot(t, signal)
    plt.vlines (beats, -1, 1, color = "black")
    plt.xlabel ("Time(sec)", fontsize = 16)
    plt.ylabel ("signal", fontsize = 16)
    plt.title("Estimated BPM", fontsize = 20)
    
    plt.show()



In [ ]:
#Παρακάτω θα απεικονίσουμε με γραμμές τα σημεία του τραγουδιού στα οποία αντιστοιχεί το τέμπο. Δηλαδή, τα σημεία που "κυριαρχεί" ο ρυθμός που βρήκαμε παραπάνω

In [ ]:
plot_tempo(foxtrot_signal_1, tempo, fs)

In [ ]:
#Για την ακουστική επιβεβαίωση θα κάνουμε αναπαραγωγή του κομματιού και θα προσθέσουμε με τη συνάρτηση clicks της librosa ένα click ανάλογα με το tempo που βρήκαμε πριν ώστε να επιβεβαιώσουμε ακουστικά το αποτέλεσμά μας.

In [ ]:
def play_tempo (signal, tempo, fs) :
    
    T = len(signal)/float(fs)
    beatfrequency = 60.0 / tempo
    beats = np.arange(0, T, beatfrequency)
    
    clicks = librosa.clicks (beats, fs, length = len(signal))
    mix = signal + clicks
    sd.play(mix, fs)

In [ ]:
play_tempo(foxtrot_signal_1, tempo, fs)


In [ ]:
#Ερώτημα 4.6
#Δημιουργήσαμε μια συνάρτηση που περιέχει τις συναρτήσεις που χρησιμοποιήθηκαν στα προηγούμενα ερωτήματα και ώστε να μπορέσουμε να κάνουμε αντίστοιχη επεξεργασία και σε άλλα κομμάτια

In [ ]:
def song_proseccing (signal,fs) :
    
    #δημιουργία διαγράμματος και συνιστωσών
    signal_plot(signal, fs)
    coefs = pywt.wavedec(signal, "db4", level=7)
    yi_norm = normalization(coefs)
    
    #δημιουργία περιβαλλουσών
    zi = absolute(yi_norm)
    surrounds = list_a_surrounds(zi)
    centralized_surrounds = central(surrounds)
    coefficients_and_surrounds_plot (centralized_surrounds, yi_norm)
    
    #υπολογισμός αυτοσυσχέτισης
    sum_surround = add_surround(centralized_surrounds, 2)
    autocorrelation = autocorr_plot(sum_surround)
    
    #υπολογισμός και επαλήθευση του tempo
    tempo = make_bpm_plot(autocorrelation,fs, signal)
    plot_tempo(signal, tempo, fs)
    play_tempo(signal, tempo, fs)


In [ ]:
#Για το δεύτερο κομμάτι foxtrot

In [ ]:
sr=22050 #sampling frequency
foxtrot_2, fs = audiosignal('foxtrot_excerpt2.wav',sr) 

foxtrot_signal_2 = foxtrot_2[(1*(2**16)):(2*(2**16))] 
sd.play(foxtrot_2,sr)

In [ ]:
song_proseccing(foxtrot_signal_2, fs)

In [ ]:
#Για το κομμάτι salsa_excerpt.wav

In [ ]:
sr=22050 #sampling frequency
salsa, fs = audiosignal('salsa_excerpt.wav',sr) 

salsa_signal = salsa[(2*(2**16)):(3*(2**16))] 
sd.play(salsa_signal,sr)

In [ ]:
song_proseccing(salsa_signal, fs)

In [ ]:
#Στο κομμάτι της salsa παρατηρούμε πολύ πιο έντονη αυτοσυχέτιση σε σχέση με τα δύο κομμάτια με αποτέλεσμα να εντοπίζουμε και μικρότερο tempo σε BPM. Το tempo που βρήκαμε είναι πάλι αρκετά κοντά στο πραγματικό tempo του τραγουδιού